# **Preprocessing for BERT model**
> [!IMPORTANT]
> RUN THE ``basic_eda.ipynb`` first

In [2]:
import pandas as pd
df = pd.read_pickle('df_for_bert.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20777 entries, 0 to 20812
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   link              20777 non-null  object
 1   publication_date  20777 non-null  object
 2   title             20777 non-null  object
 3   content           20777 non-null  object
 4   main_tag          20777 non-null  object
 5   source            20777 non-null  object
 6   content_length    20777 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


In [3]:
df['content_normalized'] = df['content'].str.normalize('NFC')
df['main_tag_normalized'] = df['main_tag'].str.normalize('NFC')

print(df['content_normalized'].head(5))

0    Trưa 4.11, quyền Chủ tịch Liên đoàn Bóng đá Ma...
1    Vụ việc FIFA bác đơn kháng cáo và giữ nguyên á...
2    Nhà phê bình bóng đá Datuk Pekan Ramli đã bày ...
3    Cựu tiền đạo Safee Sali - biểu tượng một thời ...
4    Theo nhà báo Zulhelmi Zainal Azam của Astro Ar...
Name: content_normalized, dtype: object


In [4]:
labels_list = [
    "Thể thao",
    "Thế giới",
    "Giáo dục",
    "Kinh tế",
    "Chính trị",
    "Sức khỏe",
    "Thời sự"
]

labels_mapping = {label: i for i, label in enumerate(labels_list)}

print("Label mapping:")
print(labels_mapping)

df['label_encoded'] = df['main_tag_normalized'].map(labels_mapping)

# Ensure all tags is appeared
nan_count = df['label_encoded'].isnull().sum()

if nan_count > 0:
    print(f"NaN: {nan_count}")
    all_labels_in_data = set(df['main_tag'].unique())
    label_in_map = set(labels_mapping.keys())
    unmapped_labels = all_labels_in_data - label_in_map

print("\nAfter encoding:")
print(df[['main_tag', 'label_encoded']].head(10))

print("\nTags distribution check:")
print(df['label_encoded'].value_counts().sort_index())


Label mapping:
{'Thể thao': 0, 'Thế giới': 1, 'Giáo dục': 2, 'Kinh tế': 3, 'Chính trị': 4, 'Sức khỏe': 5, 'Thời sự': 6}

After encoding:
   main_tag  label_encoded
0  Thể thao              0
1  Thể thao              0
2  Thể thao              0
3  Thể thao              0
4  Thể thao              0
5  Thể thao              0
6  Thể thao              0
7  Thể thao              0
8  Thể thao              0
9  Thể thao              0

Tags distribution check:
label_encoded
0    3032
1    2989
2    2987
3    2974
4    2959
5    2953
6    2883
Name: count, dtype: int64


In [10]:
# # Tokenization
# from pyvi import ViTokenizer

# df['content_segmented'] = df['content_normalized'].apply(lambda x: ViTokenizer.tokenize(str(x)))

# X = df.drop(columns=['label_encoded', 'main_tag', 'main_tag_normalized', 'content'])

import random

print("="*50)
print("KIỂM TRA KẾT QUẢ TÁCH TỪ (WORD SEGMENTATION)")
print("="*50)

# 1. Kiểm tra cấu trúc dữ liệu
print(f"Kích thước tập dữ liệu: {df.shape}")
print(f"Các cột hiện có: {df.columns.tolist()}")

# 2. So sánh trực quan Trước và Sau
# Lấy ngẫu nhiên 1 dòng để soi
random_idx = random.choice(df.index)
sample_original = df['content_normalized'][random_idx]
sample_segmented = df['content_segmented'][random_idx]

print(f"\n--- Mẫu ngẫu nhiên tại index {random_idx} ---")
print(f"[Gốc (Normalized)]: \n{sample_original[:200]}...") # In 200 ký tự đầu
print("-" * 20)
print(f"[Đã tách từ (Segmented)]: \n{sample_segmented[:200]}...")

# 3. Kiểm tra nhanh các từ ghép đặc trưng
# Nếu tách từ đúng, các từ này phải có dấu gạch dưới
test_words = ["thủ_tướng", "hôm_nay", "bóng_đá", "gia_đình"]
print(f"\n--- Kiểm tra sự xuất hiện của từ ghép (dấu '_') ---")
found_count = 0
for text in df['content_segmented'].head(100): # Kiểm tra 100 bài đầu
    for word in test_words:
        if word in text:
            print(f"✓ Phát hiện từ ghép chuẩn '{word}' trong bài viết.")
            found_count += 1
            break # Tìm thấy 1 từ là đủ để tin tưởng
    if found_count > 2: break 

if found_count == 0:
    print("⚠ CẢNH BÁO: Không tìm thấy các từ ghép phổ biến. Hãy kiểm tra lại thư viện pyvi.")
else:
    print("=> Quy trình tách từ hoạt động ổn định.")

KIỂM TRA KẾT QUẢ TÁCH TỪ (WORD SEGMENTATION)
Kích thước tập dữ liệu: (20777, 11)
Các cột hiện có: ['link', 'publication_date', 'title', 'content', 'main_tag', 'source', 'content_length', 'content_normalized', 'main_tag_normalized', 'label_encoded', 'content_segmented']

--- Mẫu ngẫu nhiên tại index 20602 ---
[Gốc (Normalized)]: 
AFP ngày 18.6 dẫn thông cáo của Nhà Trắng cho biết Tổng thống Mỹ Donald Trump sẽ gia hạn thêm 90 ngày để công ty mẹ ByteDance (Trung Quốc) thoái vốn khỏi ứng dụng chia sẻ video TikTok.

"Tổng thống Tr...
--------------------
[Đã tách từ (Segmented)]: 
AFP ngày 18.6 dẫn thông_cáo của Nhà_Trắng cho biết Tổng_thống Mỹ Donald_Trump sẽ gia_hạn thêm 90 ngày để công_ty_mẹ ByteDance ( Trung_Quốc ) thoái vốn khỏi ứng_dụng chia_sẻ video TikTok . 
 
 " Tổng_t...

--- Kiểm tra sự xuất hiện của từ ghép (dấu '_') ---
✓ Phát hiện từ ghép chuẩn 'bóng_đá' trong bài viết.
✓ Phát hiện từ ghép chuẩn 'bóng_đá' trong bài viết.
✓ Phát hiện từ ghép chuẩn 'bóng_đá' trong bài viết.
=> Q

In [11]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['label_encoded', 'main_tag', 'main_tag_normalized'])
y = df['label_encoded']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print(f"Sum of starting samples: {len(df)}")
print("-" * 30)
print(f"The number of samples in train: {len(X_train)} ({len(X_train)/len(df):.0%})")
print(f"The number of samples in validation: {len(X_val)} ({len(X_val)/len(df):.0%})")
print(f"The number of samples in test: {len(X_test)} ({len(X_test)/len(df):.0%})")

print("\nCheck the tag distribution in train:")
print(y_train.value_counts(normalize=True).sort_index())

print("\nCheck the tag distribution in validation:")
print(y_val.value_counts(normalize=True).sort_index())

print("\nCheck the tag distribution in validation:")
print(y_test.value_counts(normalize=True).sort_index())

Sum of starting samples: 20777
------------------------------
The number of samples in train: 16621 (80%)
The number of samples in validation: 2078 (10%)
The number of samples in test: 2078 (10%)

Check the tag distribution in train:
label_encoded
0    0.145960
1    0.143854
2    0.143794
3    0.143132
4    0.142410
5    0.142109
6    0.138740
Name: proportion, dtype: float64

Check the tag distribution in validation:
label_encoded
0    0.145813
1    0.143888
2    0.143407
3    0.143407
4    0.142445
5    0.141963
6    0.139076
Name: proportion, dtype: float64

Check the tag distribution in validation:
label_encoded
0    0.145813
1    0.143888
2    0.143888
3    0.142926
4    0.142445
5    0.142445
6    0.138595
Name: proportion, dtype: float64


In [12]:
import os

os.makedirs('train', exist_ok=True)
os.makedirs('val', exist_ok=True)
os.makedirs('test', exist_ok=True)

X_train.to_csv('./train/X_train_bert.csv', index=False, encoding='utf-8')
y_train.to_frame().to_csv('./train/y_train_bert.csv', index=False, encoding='utf-8')

X_val.to_csv('./val/X_val_bert.csv', index=False, encoding='utf-8')
y_val.to_frame().to_csv('./val/y_val_bert.csv', index=False, encoding='utf-8')

X_test.to_csv('./test/X_test_bert.csv', index=False, encoding='utf-8')
y_test.to_frame().to_csv('./test/y_test_bert.csv', index=False, encoding='utf-8')

print("\nCreated final dataset.")


Created final dataset.
